In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 0


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization

In [5]:
optimizer_init_file_name = "optimizer/optimizer_init.json"

In [6]:
ax_client_init = sdl.initialize_ax()
ax_client_init.save_to_json_file(optimizer_init_file_name)

[WARNING 08-05 13:31:34] ax.service.utils.instantiation: Objective thresholds were not set. They will be selected using a heuristic, but should be specified on the objective for best performance.


# Generate recommendations

In [7]:
time_start = time.time()

# generate recomms
ax_client_init = AxClient.load_from_json_file(optimizer_init_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=0)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

[INFO 08-05 13:31:34] ax.service.ax_client: Generated new trial 0 with parameters {'Drug': 48, 'SL_1': 59, 'SL_2': 49, 'SL_3': 31, 'LL_1': 96, 'LL_2': 8, 'LL_3': 10, 'P_1': 35, 'P_2': 86, 'P_3': 76, 'S_1': 88, 'S_2': 88, 'S_3': 81, 'Water': 85} using model Sobol.
[INFO 08-05 13:31:34] ax.service.ax_client: Generated new trial 1 with parameters {'Drug': 93, 'SL_1': 38, 'SL_2': 50, 'SL_3': 84, 'LL_1': 49, 'LL_2': 66, 'LL_3': 78, 'P_1': 52, 'P_2': 25, 'P_3': 20, 'S_1': 36, 'S_2': 5, 'S_3': 29, 'Water': 0} using model Sobol.
[INFO 08-05 13:31:34] ax.service.ax_client: Generated new trial 2 with parameters {'Drug': 51, 'SL_1': 77, 'SL_2': 13, 'SL_3': 24, 'LL_1': 20, 'LL_2': 94, 'LL_3': 26, 'P_1': 92, 'P_2': 2, 'P_3': 28, 'S_1': 16, 'S_2': 31, 'S_3': 59, 'Water': 53} using model Sobol.
[INFO 08-05 13:31:34] ax.service.ax_client: Generated new trial 3 with parameters {'Drug': 9, 'SL_1': 20, 'SL_2': 88, 'SL_3': 72, 'LL_1': 73, 'LL_2': 36, 'LL_3': 61, 'P_1': 20, 'P_2': 63, 'P_3': 75, 'S_1': 58,

0 min


In [8]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,0.099000,0.099125,0.101188,0.098000,0.096875,0.100312,0.101875,0.098750,0.104375,0.100437,0.255875,0.243563,0.254500,0.245938
std,4.760952,0.054264,0.060421,0.061329,0.060829,0.054217,0.058191,0.063191,0.057953,0.065646,0.061886,0.138794,0.125753,0.146358,0.150867
min,0.000000,0.008000,0.017000,0.004000,0.000000,0.003000,0.002000,0.000000,0.004000,0.005000,0.013000,0.019000,0.049000,0.031000,0.000000
25%,3.750000,0.054000,0.051750,0.061500,0.054500,0.046500,0.059000,0.051000,0.066500,0.045750,0.058000,0.154250,0.168750,0.149500,0.163250
50%,7.500000,0.109500,0.094000,0.094000,0.089500,0.103000,0.098500,0.110500,0.084500,0.110500,0.095500,0.258500,0.252000,0.247000,0.236500
75%,11.250000,0.140750,0.148500,0.143000,0.140000,0.129750,0.132250,0.150750,0.141250,0.161500,0.140500,0.351750,0.354750,0.373500,0.325500
max,15.000000,0.175000,0.196000,0.202000,0.221000,0.193000,0.220000,0.198000,0.215000,0.210000,0.248000,0.514000,0.485000,0.473000,0.603000


In [9]:
unlabeled_trial
unlabeled_trial.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000,16.000000,16.00000,16.000000,16.000000
mean,7.500000,50.562500,50.062500,50.062500,49.875000,49.937500,50.687500,49.937500,50.00000,50.062500,50.000000,49.937500,50.06250,50.000000,50.250000
std,4.760952,30.106409,29.862951,30.200373,30.210098,29.949889,30.341872,30.573341,29.98444,30.387429,29.754551,29.793665,29.50812,29.783664,30.105371
min,0.000000,3.000000,5.000000,2.000000,0.000000,1.000000,1.000000,0.000000,2.00000,2.000000,6.000000,5.000000,5.00000,4.000000,0.000000
25%,3.750000,26.750000,26.000000,24.500000,27.750000,26.750000,28.500000,25.250000,25.25000,24.250000,26.000000,26.000000,28.25000,26.250000,29.500000
50%,7.500000,49.500000,49.000000,49.500000,48.500000,51.500000,51.000000,50.000000,49.50000,50.500000,49.000000,52.000000,50.50000,50.500000,48.500000
75%,11.250000,74.500000,74.750000,75.500000,74.250000,74.250000,72.750000,74.250000,73.25000,74.000000,75.250000,71.250000,72.00000,73.500000,74.500000
max,15.000000,100.000000,98.000000,98.000000,96.000000,96.000000,99.000000,100.000000,96.00000,99.000000,98.000000,99.000000,94.00000,97.000000,97.000000


In [10]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000,16.0000,16.00000,16.000000,16.000000,16.000000,16.000000,16.000000,16.000000
mean,7.500000,34.650000,34.693750,35.415625,34.300000,33.906250,35.109375,35.656250,34.5625,36.53125,35.153125,255.875000,243.562500,254.500000,245.937500,0.021875
std,4.760952,18.992244,21.147213,21.465101,21.290052,18.975843,20.366960,22.116705,20.2837,22.97624,21.660059,138.793792,125.753181,146.357553,150.867036,0.298869
min,0.000000,2.800000,5.950000,1.400000,0.000000,1.050000,0.700000,0.000000,1.4000,1.75000,4.550000,19.000000,49.000000,31.000000,0.000000,-0.700000
25%,3.750000,18.900000,18.112500,21.525000,19.075000,16.275000,20.650000,17.850000,23.2750,16.01250,20.300000,154.250000,168.750000,149.500000,163.250000,0.000000
50%,7.500000,38.325000,32.900000,32.900000,31.325000,36.050000,34.475000,38.675000,29.5750,38.67500,33.425000,258.500000,252.000000,247.000000,236.500000,0.000000
75%,11.250000,49.262500,51.975000,50.050000,49.000000,45.412500,46.287500,52.762500,49.4375,56.52500,49.175000,351.750000,354.750000,373.500000,325.500000,0.350000
max,15.000000,61.250000,68.600000,70.700000,77.350000,67.550000,77.000000,69.300000,75.2500,73.50000,86.800000,514.000000,485.000000,473.000000,603.000000,0.350000


In [11]:
dead_volume = 2.5  # mL, for 20 mL vials

print(f"For 3 mm aspiration height, dead volume of 20 mL vials is around {dead_volume} mL") 

round(transfer[['Drug', 'SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000, 2)

For 3 mm aspiration height, dead volume of 20 mL vials is around 2.5 mL


Drug       0.55
SL_1       0.56
SL_2       0.57
SL_3       0.55
LL_1       0.54
LL_2       0.56
LL_3       0.57
P_1        0.55
P_2        0.58
P_3        0.56
S_1        4.09
S_2        3.90
S_3        4.07
Water      3.94
Solvent    0.00
dtype: float64

In [12]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"
unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"
unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"
transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [13]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [14]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [15]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [16]:
updated_script

('protocol/iteration_0_OT_2_protocol_1.py',
 'protocol/iteration_0_OT_2_protocol_2.py')

# Perform experiment

# Results

In [17]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,0,0.096,0.118,0.098,0.062,0.193,0.016,0.020,0.070,0.173,0.153,0.257,0.257,0.237,0.249,12,0
1,1,0.168,0.068,0.090,0.151,0.088,0.119,0.141,0.094,0.045,0.036,0.514,0.071,0.414,0.000,12,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.061,0.215,0.005,0.066,0.101,0.195,0.371,0.333,12,0
3,3,0.017,0.039,0.170,0.139,0.141,0.070,0.118,0.039,0.122,0.145,0.341,0.371,0.065,0.224,12,0
4,4,0.048,0.196,0.158,0.109,0.114,0.051,0.103,0.162,0.046,0.013,0.052,0.364,0.461,0.123,12,0
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.159,0.025,0.159,0.248,0.301,0.174,0.082,0.443,11,0
6,6,0.119,0.116,0.138,0.143,0.045,0.122,0.019,0.074,0.143,0.082,0.372,0.056,0.381,0.190,12,0
7,7,0.069,0.056,0.024,0.085,0.155,0.091,0.198,0.131,0.099,0.091,0.169,0.360,0.173,0.298,12,0
8,8,0.056,0.165,0.198,0.020,0.077,0.060,0.083,0.173,0.028,0.139,0.247,0.485,0.077,0.191,12,0
9,9,0.155,0.030,0.004,0.116,0.170,0.185,0.136,0.004,0.136,0.064,0.345,0.049,0.283,0.323,12,0


In [18]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Good,A1,122.332985,0.365896
1,Good,A2,134.430167,0.389210
2,Good,A3,128.720083,0.463264
3,Good,A4,287.326856,0.328742
4,Good,A5,305.815598,0.292998
5,Good,A6,364.323470,0.448564
6,Good,B1,227.511620,0.168821
7,Good,B2,227.907360,0.124415
8,Good,B3,231.810253,0.181534
9,Good,B4,267.696335,0.334484


In [19]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,128.494412,6.051748,0.406123,0.050840,1
1,Formulation 2,319.155308,40.194285,0.356768,0.081482,1
2,Formulation 3,229.076411,2.375830,0.158257,0.029989,1
3,Formulation 4,278.574059,15.383424,0.327699,0.009595,1
4,Formulation 5,251.194083,0.117534,0.240794,0.013777,1
5,Formulation 6,196.995001,4.055198,0.190406,0.020802,1
6,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,Formulation 8,218.697531,10.716824,0.270794,0.029588,1
8,Formulation 9,263.040616,3.837711,0.297712,0.013304,1
9,Formulation 10,214.784726,5.099425,0.255007,0.030796,1


In [20]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,0,0.096,0.118,0.098,0.062,0.193,0.016,0.020,0.070,0.173,...,0.237,0.249,12,0,Formulation 1,128.494412,6.051748,0.406123,0.050840,1
1,1,0.168,0.068,0.090,0.151,0.088,0.119,0.141,0.094,0.045,...,0.414,0.000,12,0,Formulation 2,319.155308,40.194285,0.356768,0.081482,1
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.061,0.215,0.005,...,0.371,0.333,12,0,Formulation 3,229.076411,2.375830,0.158257,0.029989,1
3,3,0.017,0.039,0.170,0.139,0.141,0.070,0.118,0.039,0.122,...,0.065,0.224,12,0,Formulation 4,278.574059,15.383424,0.327699,0.009595,1
4,4,0.048,0.196,0.158,0.109,0.114,0.051,0.103,0.162,0.046,...,0.461,0.123,12,0,Formulation 5,251.194083,0.117534,0.240794,0.013777,1
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.159,0.025,0.159,...,0.082,0.443,11,0,Formulation 6,196.995001,4.055198,0.190406,0.020802,1
6,6,0.119,0.116,0.138,0.143,0.045,0.122,0.019,0.074,0.143,...,0.381,0.190,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0
7,7,0.069,0.056,0.024,0.085,0.155,0.091,0.198,0.131,0.099,...,0.173,0.298,12,0,Formulation 8,218.697531,10.716824,0.270794,0.029588,1
8,8,0.056,0.165,0.198,0.020,0.077,0.060,0.083,0.173,0.028,...,0.077,0.191,12,0,Formulation 9,263.040616,3.837711,0.297712,0.013304,1
9,9,0.155,0.030,0.004,0.116,0.170,0.185,0.136,0.004,0.136,...,0.283,0.323,12,0,Formulation 10,214.784726,5.099425,0.255007,0.030796,1


In [21]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.096,0.118,0.098,0.062,0.193,0.016,0.020,0.070,0.173,...,12,0,Formulation 1,128.494412,6.051748,0.406123,0.050840,1,192.0,0
1,1,0.168,0.068,0.090,0.151,0.088,0.119,0.141,0.094,0.045,...,12,0,Formulation 2,319.155308,40.194285,0.356768,0.081482,1,336.0,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.061,0.215,0.005,...,12,0,Formulation 3,229.076411,2.375830,0.158257,0.029989,1,238.0,0
3,3,0.017,0.039,0.170,0.139,0.141,0.070,0.118,0.039,0.122,...,12,0,Formulation 4,278.574059,15.383424,0.327699,0.009595,1,34.0,0
4,4,0.048,0.196,0.158,0.109,0.114,0.051,0.103,0.162,0.046,...,12,0,Formulation 5,251.194083,0.117534,0.240794,0.013777,1,96.0,0
5,5,0.175,0.025,0.063,0.000,0.003,0.142,0.159,0.025,0.159,...,11,0,Formulation 6,196.995001,4.055198,0.190406,0.020802,1,350.0,0
6,6,0.119,0.116,0.138,0.143,0.045,0.122,0.019,0.074,0.143,...,12,0,Formulation 7,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
7,7,0.069,0.056,0.024,0.085,0.155,0.091,0.198,0.131,0.099,...,12,0,Formulation 8,218.697531,10.716824,0.270794,0.029588,1,138.0,0
8,8,0.056,0.165,0.198,0.020,0.077,0.060,0.083,0.173,0.028,...,12,0,Formulation 9,263.040616,3.837711,0.297712,0.013304,1,112.0,0
9,9,0.155,0.030,0.004,0.116,0.170,0.185,0.136,0.004,0.136,...,12,0,Formulation 10,214.784726,5.099425,0.255007,0.030796,1,310.0,0


In [22]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,1,0.128494,0.006052,0.096,0.0,0.406123,0.050840,1.000000
1,1,0.319155,0.040194,0.168,0.0,0.356768,0.081482,1.000000
2,1,0.229076,0.002376,0.119,0.0,0.158257,0.029989,1.000000
3,1,0.278574,0.015383,0.017,0.0,0.327699,0.009595,1.000000
4,1,0.251194,0.000118,0.048,0.0,0.240794,0.013777,1.000000
5,1,0.196995,0.004055,0.175,0.0,0.190406,0.020802,0.916667
6,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,1.000000
7,1,0.218698,0.010717,0.069,0.0,0.270794,0.029588,1.000000
8,1,0.263041,0.003838,0.056,0.0,0.297712,0.013304,1.000000
9,1,0.214785,0.005099,0.155,0.0,0.255007,0.030796,1.000000


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,0,0.096,0.118,0.098,0.062,0.193,0.016,0.020,0.070,0.173,...,1.0,0,Formulation 1,0.128494,0.006052,0.406123,0.050840,1,0.096,0
1,1,0.168,0.068,0.090,0.151,0.088,0.119,0.141,0.094,0.045,...,1.0,0,Formulation 2,0.319155,0.040194,0.356768,0.081482,1,0.168,0
2,2,0.119,0.180,0.030,0.056,0.047,0.220,0.061,0.215,0.005,...,1.0,0,Formulation 3,0.229076,0.002376,0.158257,0.029989,1,0.119,0
3,3,0.017,0.039,0.170,0.139,0.141,0.070,0.118,0.039,0.122,...,1.0,0,Formulation 4,0.278574,0.015383,0.327699,0.009595,1,0.017,0
4,4,0.048,0.196,0.158,0.109,0.114,0.051,0.103,0.162,0.046,...,1.0,0,Formulation 5,0.251194,0.000118,0.240794,0.013777,1,0.048,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 0 with data: {'Solu': (0.096, 0), 'Size': (0.128494, 0.006052), 'PDI': (0.406123, 0.05084)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 1 with data: {'Solu': (0.168, 0), 'Size': (0.319155, 0.040194), 'PDI': (0.356768, 0.081482)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 2 with data: {'Solu': (0.119, 0), 'Size': (0.229076, 0.002376), 'PDI': (0.158257, 0.029989)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 3 with data: {'Solu': (0.017, 0), 'Size': (0.278574, 0.015383), 'PDI': (0.327699, 0.009595)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 4 with data: {'Solu': (0.048, 0), 'Size': (0.251194, 0.000118), 'PDI': (0.240794, 0.013777)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 5 with data: {'Solu': (0.175, 0), 'Size': (0.196995, 0.004055), 'PDI': (0.190406, 0.020802)}.
[INFO 08-05 16:03:32] ax.service.ax_client: Completed trial 6 with data: {'So